In [ ]:
%cd ../../src

In [1]:
import kgbench
data = kgbench.load('dmg777k', final= True, torch= True)

c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded data dmg777k (55.85s).


In [4]:
data.name = data.name+"_2"

In [6]:
import pandas as pd
df = pd.DataFrame(columns=["h","r","t"])
        
dfs = []
for d in data.triples:
    dfs.append(pd.DataFrame(
        [[
            data.i2e[d[0]][0],
            data.i2r[d[1]],
            data.i2e[d[2]][0]
        ]],
        columns=["h","r","t"]
                   
    ))
df = pd.concat(dfs,ignore_index=True)
df.to_csv("data.tsv.gz", 
    index=False,
    sep="\t" ,
    compression="gzip")
        

In [7]:
import os

In [12]:
not os.path.exists('data.tsv.gz')

False

In [5]:
data.name

'dmg777k_2'

In [6]:
import pandas as pd
df = pd.DataFrame(columns=["h","r","t"])

In [7]:
dfs = []
for d in data.triples:
    dfs.append(pd.DataFrame(
        [[
        data.i2e[d[0]][0],
        data.i2r[d[1]],
        data.i2e[d[2]][0]
        ]],
        columns=["h","r","t"]
    ))
df = pd.concat(dfs,ignore_index=True)


KeyboardInterrupt: 

In [4]:
df.to_csv("dmg777k.tsv.gz", 
           index=False,
           sep="\t" ,
           compression="gzip")

In [1]:
from pykeen.triples import TriplesFactory
dat = TriplesFactory.from_path("dmg777k.tsv.gz")

c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train,valid,test = dat.split([0.85,0.149,0.001])

using automatically assigned random_state=2968855886


In [6]:
from pykeen.pipeline import pipeline
result = pipeline(
    training= train,
    validation= valid,
    testing = test,
    model = "TransE"

)

No random seed is specified. Setting to 3551894453.
Training epochs on cuda:0: 100%|██████████| 5/5 [02:26<00:00, 29.21s/epoch, loss=0.0504, prev_loss=0.081]
INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...


KeyboardInterrupt: 

In [8]:
from pykeen.utils import get_preferred_device
get_preferred_device(model)

device(type='cpu')

In [11]:
from pykeen.utils import resolve_device
resolve_device('gpu')
#get_devices(model)

device(type='cuda')

In [13]:
dat.entity_id_to_label

{0: '',
 1: '"\'t Hof Alle Morgen Nieuwe Zorgen". Bakstenen woonhuis met pannen zadeldak. Tuittopgevels met vlechtingen en jaarankers 1796. Bakgoot op klossen. Dicht bijeen liggende staafankers in langsgevel. 6-ruits schuiframen in topgevel en gewijzigde eenruits schuiframen in voorgevel. Rollagen boven ramen. Oude voordeur met nieuwe omlijsting. Houten hek langs voorgevel (langsgevel). Schuurtje in houten met pannen zadeldak.',
 2: '"\'t Munnikenhof". OVERBLIJFSEL van een XVII A tot buitenplaats verbouwde laatmiddeleeuwse UITHOF van de Middelburgse abdij. Het na de sloping van de hoofdvleugel ong 1860 overgebleven gedeelte dat gespaard bleef als jachthuis omvat de in XVII aangebouwde vleugel voorzien van een gevel van breuksteen, aan de rechterzijde geflankeerd door de (verminkte) trap-uitzicht-toren. XVII fragmenten, o.a. gebeeldhouwde balkkoppen. De XIX B gevel en de toren bepleisterd.',
 3: '"Aazicht". Zeer fraai witgepleisterde boerenwoning, dwarstype met hoge opkamer, rieten dak,

In [19]:
data.training.numpy()

array([[282346,      1],
       [276422,      4],
       [277418,      1],
       ...,
       [277014,      1],
       [275650,      1],
       [273554,      1]], dtype=int64)

In [20]:

new_id = []
for t in data.training.numpy():
    new_id.append(dat.entity_to_id[data.i2e[t[0]][0]])

In [25]:
import numpy as np
np.array(new_id)
aaa = data.training.clone()

In [26]:
aaa

tensor([[282346,      1],
        [276422,      4],
        [277418,      1],
        ...,
        [277014,      1],
        [275650,      1],
        [273554,      1]])

In [28]:
import torch
aaa[:,0] = torch.tensor(np.array(new_id))

In [40]:
bbb = data.withheld.clone()

In [41]:
new_id = []
for t in data.withheld.numpy():
    new_id.append(dat.entity_to_id[data.i2e[t[0]][0]])

In [42]:
bbb[:,0] = torch.tensor(np.array(new_id))

In [29]:
aaa

tensor([[276535,      1],
        [273573,      4],
        [274071,      1],
        ...,
        [273869,      1],
        [273187,      1],
        [272139,      1]])

In [31]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
class SVM():
    def __init__(self, random_state=42, n_estimators=[10, 20, 40], max_depth=[3, 5, None], cv=10):
        svc = SVC(random_state=random_state)
        self.classifier = GridSearchCV(svc, {"C": [10**i for i in range(-3, 4)]}, cv=cv)

    def fit(self, train_embeddings, train_target):
        return self.classifier.fit(train_embeddings, train_target)

    def predict(self, test_embeddings):
        return self.classifier.predict(test_embeddings)
svm = SVM()

In [34]:
len(data.training,aaa)

5394

In [33]:
len(result.model.entity_representations[0]().detach().cpu().numpy())

332197

In [37]:
embeddings = result.model.entity_representations[0]().detach().cpu().numpy()

In [38]:
embeddings[aaa[:,0]]

array([[ 0.01884923, -0.1427332 ,  0.03336553, ...,  0.10639732,
        -0.01507117, -0.0994466 ],
       [-0.20989619,  0.13364498, -0.09089275, ..., -0.02827386,
        -0.22132973, -0.14256579],
       [-0.25994316,  0.2446012 ,  0.18166216, ...,  0.07910308,
        -0.06110276, -0.29342347],
       ...,
       [-0.08455671, -0.13026743,  0.14201468, ..., -0.06386667,
        -0.11587907, -0.21550117],
       [-0.16722633, -0.17334956, -0.10072945, ...,  0.02343123,
        -0.0991317 ,  0.15166669],
       [-0.26240227, -0.06625596, -0.17136721, ..., -0.09383324,
        -0.10612053, -0.12098309]], dtype=float32)

In [39]:
svm.fit(embeddings[aaa[:,0]],aaa[:,1])

GridSearchCV(cv=10, estimator=SVC(random_state=42),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

In [44]:
svm.predict(embeddings[bbb[:,0]])

array([1, 1, 1, ..., 3, 1, 1], dtype=int64)

In [45]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [46]:
predictions = svm.predict(embeddings[bbb[:,0]])

print(
    f"Predicted {len(embeddings[bbb[:,0]])} entities with an accuracy of "
    + f"{accuracy_score(bbb[:,1], predictions) * 100 :.4f}%"
)
print("Confusion Matrix :")
print(confusion_matrix(bbb[:,1], predictions))

Predicted 2001 entities with an accuracy of 47.5262%
Confusion Matrix :
[[  0  58   0  18   0]
 [  0 849   0 107   0]
 [  0 119   0  25   0]
 [  0 453   0 102   0]
 [  0 213   0  57   0]]


In [16]:
data.i2e[282346][0]

'https://data.labs.pdok.nl/rce/id/monument/36903'

In [18]:
dat.entity_to_id[data.i2e[282346][0]]

276535

In [22]:
r = pipeline(
    training= train,
    validation= valid,
    testing = test,
    model = "TransE",
    evaluator="classification",
    evaluation_kwargs={"additional_filter_triples":[train.mapped_triples]}


)

INFO:pykeen.pipeline.api:Using device: None
Training epochs on cuda:0: 100%|██████████| 5/5 [02:40<00:00, 32.09s/epoch, loss=0.0497, prev_loss=0.0788]
INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\rexmex\metrics\classification.py:484: RuntimeWarning: invalid value encountered in power
  pthr = (fpr**0.5) / (fpr**0.5 + tpr**0.5)
c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\rexmex\metrics\classification.py:547: RuntimeWarning: invalid value encountered in power
  fm = (ppv * tpr) ** 0.5
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=64.


ValueError: need at least one array to concatenate

In [13]:
from pykeen.datasets import Nations

In [15]:
Nations().training.mapped_triples

tensor([[ 0,  3,  2],
        [ 0,  3,  3],
        [ 0,  3, 10],
        ...,
        [13, 54,  9],
        [13, 54, 11],
        [13, 54, 12]])

In [17]:
Nations().training.triples

array([['brazil', 'blockpositionindex', 'china'],
       ['brazil', 'blockpositionindex', 'cuba'],
       ['brazil', 'blockpositionindex', 'poland'],
       ...,
       ['ussr', 'weightedunvote', 'netherlands'],
       ['ussr', 'weightedunvote', 'uk'],
       ['ussr', 'weightedunvote', 'usa']], dtype='<U19')

In [23]:
train.mapped_triples

tensor([[129881,     60, 328168],
        [130003,     31, 201144],
        [130004,     31, 201144],
        ...,
        [224406,      7,   3948],
        [274021,     40, 270846],
        [134858,     31, 201144]])

In [59]:
from pykeen.models import TransE
m = TransE(triples_factory = train)

In [60]:
m.entity_representations[0]().detach().cpu().numpy()

array([[ 0.19300215, -0.09616999,  0.06986993, ...,  0.20166168,
        -0.16835561,  0.18663397],
       [-0.21628156, -0.19624296,  0.05492296, ..., -0.05595579,
         0.04201833, -0.22109382],
       [ 0.0042635 ,  0.03377317,  0.13749936, ...,  0.0110116 ,
        -0.22750352, -0.18205278],
       ...,
       [-0.05933924, -0.18856871, -0.0646616 , ...,  0.02946395,
         0.21922573, -0.11095158],
       [ 0.1152747 , -0.18407854, -0.16869323, ..., -0.0951511 ,
         0.20427771,  0.12166793],
       [ 0.22103404, -0.10885949,  0.13154294, ..., -0.24118991,
         0.20924802,  0.1244884 ]], dtype=float32)

In [61]:
m.train()

TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(332197, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(61, 50)
    )
  )
  (weight_regularizers): ModuleList()
)

In [63]:
m.cpu().train.

TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(332197, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(61, 50)
    )
  )
  (weight_regularizers): ModuleList()
)

In [21]:
import torch
torch.cuda.set_device(0)
print(torch.cuda)
# Get a training dataset
from pykeen.datasets import Nations
dataset = Nations()
training_triples_factory = dataset.training

# Pick a model
from pykeen.models import TransE
model = TransE(triples_factory=train)
model = model.to(resolve_device('gpu'))
#model.device = resolve_device('gpu')

# Pick an optimizer from Torch
from torch.optim import Adam
optimizer = Adam(params=model.get_grad_params())

# Pick a training approach (sLCWA or LCWA)
from pykeen.training import SLCWATrainingLoop
training_loop = SLCWATrainingLoop(
    model=model,
    triples_factory=train,
    optimizer=optimizer,
    #device= resolve_device('gpu')
)
_ = training_loop.train(
    triples_factory=train,
    num_epochs=1,
    batch_size=256,
)

<module 'torch.cuda' from 'c:\\Users\\Noctris\\.virtualenvs\\rdf-literal-preprocessing-20b3_M0v\\lib\\site-packages\\torch\\cuda\\__init__.py'>


Training epochs on cuda:0: 100%|██████████| 1/1 [00:29<00:00, 29.50s/epoch, loss=0.78, prev_loss=nan]


In [4]:
model.entity_representations[0]().detach().cpu().numpy()

array([[ 0.0607857 , -0.14597774, -0.12604184, ..., -0.15237004,
        -0.0292913 , -0.00799458],
       [-0.19499598, -0.16713472,  0.14650533, ..., -0.23348239,
         0.17337446,  0.16763677],
       [-0.2329021 ,  0.02463412,  0.19398943, ...,  0.22090901,
        -0.14801829, -0.06185253],
       ...,
       [-0.22293131,  0.00107062,  0.13410026, ...,  0.16890113,
         0.10114383, -0.1568804 ],
       [ 0.1192558 , -0.08467166, -0.17065723, ..., -0.0484582 ,
        -0.04316909,  0.11142302],
       [-0.25153634,  0.25886342,  0.04396984, ...,  0.02204386,
        -0.13480027,  0.19759332]], dtype=float32)

In [66]:
from pykeen.evaluation import RankBasedEvaluator, ClassificationEvaluator

In [67]:
evaluator = RankBasedEvaluator()

# Get triples to test
mapped_triples = dataset.testing.mapped_triples

# Evaluate
results = evaluator.evaluate(
    model=model,
    mapped_triples=train.mapped_triples,
    batch_size=1024,
    additional_filter_triples=[
        train.mapped_triples,
        valid.mapped_triples,
    ],
)

Evaluating on cpu: 100%|██████████| 201/201 [00:00<00:00, 3.07ktriple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.08s seconds


In [72]:
results.data

{('median_rank', 'head', 'optimistic'): 4.0,
 ('median_rank', 'tail', 'optimistic'): 4.0,
 ('median_rank', 'both', 'optimistic'): 4.0,
 ('median_rank', 'head', 'realistic'): 4.0,
 ('median_rank', 'tail', 'realistic'): 4.0,
 ('median_rank', 'both', 'realistic'): 4.0,
 ('median_rank', 'head', 'pessimistic'): 4.0,
 ('median_rank', 'tail', 'pessimistic'): 4.0,
 ('median_rank', 'both', 'pessimistic'): 4.0,
 ('median_absolute_deviation', 'head', 'optimistic'): 2.965204437011204,
 ('median_absolute_deviation', 'tail', 'optimistic'): 2.965204437011204,
 ('median_absolute_deviation', 'both', 'optimistic'): 2.965204437011204,
 ('median_absolute_deviation', 'head', 'realistic'): 2.965204437011204,
 ('median_absolute_deviation', 'tail', 'realistic'): 2.965204437011204,
 ('median_absolute_deviation', 'both', 'realistic'): 2.965204437011204,
 ('median_absolute_deviation', 'head', 'pessimistic'): 2.965204437011204,
 ('median_absolute_deviation', 'tail', 'pessimistic'): 2.965204437011204,
 ('median_ab

In [75]:
evaluator = ClassificationEvaluator()

# Get triples to test
mapped_triples = dataset.testing.mapped_triples

# Evaluate
results = evaluator.evaluate(
    model=model,
    mapped_triples=train.mapped_triples,
    batch_size=1024,
    additional_filter_triples=[
        train.mapped_triples,
        valid.mapped_triples,
    ],
)

Evaluating on cpu:   0%|          | 0.00/661k [00:00<?, ?triple/s]


IndexError: index out of range in self

In [74]:
results.data

{'true_negative_rate': 10.492853164672852,
 'true_positive_rate': -9.767412185668945,
 'positive_predictive_value': 0.46119391918182373,
 'negative_predictive_value': 0.5394726395606995,
 'false_negative_rate': 10.767412185668945,
 'false_positive_rate': -9.492853164672852,
 'false_discovery_rate': 0.538806140422821,
 'false_omission_rate': 0.4605273902416229,
 'positive_likelihood_ratio': 1.0289226770401,
 'negative_likelihood_ratio': 1.026166319847107,
 'prevalence_threshold': nan,
 'threat_score': 0.9381706714630127,
 'fowlkes_mallows_index': nan,
 'informedness': -0.27455902099609375,
 'markedness': 0.0006666183471679688,
 'diagnostic_odds_ratio': 1.0026860237121582,
 'roc_auc_score': 0.498152619669675,
 'accuracy_score': 0.5119047619047619,
 'balanced_accuracy_score': 0.5077596177299877,
 'f1_score': 0.4625887493173129,
 'average_precision_score': 0.42324677950616774,
 'matthews_correlation_coefficient': 0.01551923545997537}

In [22]:
a = [5,8,3]

In [27]:
import numpy as np
a = np.array(a)

In [30]:
a[[1,0,2]]

array([8, 5, 3])

In [5]:
dat.entity_id_to_label

{0: '',
 1: '"\'t Hof Alle Morgen Nieuwe Zorgen". Bakstenen woonhuis met pannen zadeldak. Tuittopgevels met vlechtingen en jaarankers 1796. Bakgoot op klossen. Dicht bijeen liggende staafankers in langsgevel. 6-ruits schuiframen in topgevel en gewijzigde eenruits schuiframen in voorgevel. Rollagen boven ramen. Oude voordeur met nieuwe omlijsting. Houten hek langs voorgevel (langsgevel). Schuurtje in houten met pannen zadeldak.',
 2: '"\'t Munnikenhof". OVERBLIJFSEL van een XVII A tot buitenplaats verbouwde laatmiddeleeuwse UITHOF van de Middelburgse abdij. Het na de sloping van de hoofdvleugel ong 1860 overgebleven gedeelte dat gespaard bleef als jachthuis omvat de in XVII aangebouwde vleugel voorzien van een gevel van breuksteen, aan de rechterzijde geflankeerd door de (verminkte) trap-uitzicht-toren. XVII fragmenten, o.a. gebeeldhouwde balkkoppen. De XIX B gevel en de toren bepleisterd.',
 3: '"Aazicht". Zeer fraai witgepleisterde boerenwoning, dwarstype met hoge opkamer, rieten dak,

In [ ]:
reorder = []
for i in dat.entity_id_to_label:
    data.e2i[]


In [8]:
dat.entity_to_id

{'': 0,
 '"\'t Hof Alle Morgen Nieuwe Zorgen". Bakstenen woonhuis met pannen zadeldak. Tuittopgevels met vlechtingen en jaarankers 1796. Bakgoot op klossen. Dicht bijeen liggende staafankers in langsgevel. 6-ruits schuiframen in topgevel en gewijzigde eenruits schuiframen in voorgevel. Rollagen boven ramen. Oude voordeur met nieuwe omlijsting. Houten hek langs voorgevel (langsgevel). Schuurtje in houten met pannen zadeldak.': 1,
 '"\'t Munnikenhof". OVERBLIJFSEL van een XVII A tot buitenplaats verbouwde laatmiddeleeuwse UITHOF van de Middelburgse abdij. Het na de sloping van de hoofdvleugel ong 1860 overgebleven gedeelte dat gespaard bleef als jachthuis omvat de in XVII aangebouwde vleugel voorzien van een gevel van breuksteen, aan de rechterzijde geflankeerd door de (verminkte) trap-uitzicht-toren. XVII fragmenten, o.a. gebeeldhouwde balkkoppen. De XIX B gevel en de toren bepleisterd.': 2,
 '"Aazicht". Zeer fraai witgepleisterde boerenwoning, dwarstype met hoge opkamer, rieten dak, 17

In [10]:
reorder = []
for e in data.i2e:
    reorder.append(dat.entity_to_id[e[0]])

In [12]:
data.i2e

[('', 'none'),
 ('"\'t Hof Alle Morgen Nieuwe Zorgen". Bakstenen woonhuis met pannen zadeldak. Tuittopgevels met vlechtingen en jaarankers 1796. Bakgoot op klossen. Dicht bijeen liggende staafankers in langsgevel. 6-ruits schuiframen in topgevel en gewijzigde eenruits schuiframen in voorgevel. Rollagen boven ramen. Oude voordeur met nieuwe omlijsting. Houten hek langs voorgevel (langsgevel). Schuurtje in houten met pannen zadeldak.',
  '@nl-nl'),
 ('"\'t Munnikenhof". OVERBLIJFSEL van een XVII A tot buitenplaats verbouwde laatmiddeleeuwse UITHOF van de Middelburgse abdij. Het na de sloping van de hoofdvleugel ong 1860 overgebleven gedeelte dat gespaard bleef als jachthuis omvat de in XVII aangebouwde vleugel voorzien van een gevel van breuksteen, aan de rechterzijde geflankeerd door de (verminkte) trap-uitzicht-toren. XVII fragmenten, o.a. gebeeldhouwde balkkoppen. De XIX B gevel en de toren bepleisterd.',
  '@nl-nl'),
 ('"Aazicht". Zeer fraai witgepleisterde boerenwoning, dwarstype me

In [13]:
len(reorder)

341270

In [15]:
from pykeen.models import TransE
from torch.optim import Adam
from pykeen.training import SLCWATrainingLoop
import pandas as pd
from pykeen.triples import TriplesFactory
from typing import List, Sequence, Tuple
from pyrdf2vec.typings import Embeddings
from pykeen.utils import resolve_device

train_loop_args={"num_epochs": 5,'batch_size':256}
model = TransE(triples_factory=dat)
model = model.to(resolve_device('gpu'))
optimizer = Adam(params=model.get_grad_params())
training_loop = SLCWATrainingLoop(
    model=model,
    triples_factory=dat,
    optimizer=optimizer,
    )
train_loop_args = train_loop_args

training_loop.train(
    triples_factory=dat,
    **train_loop_args
        )
embeddings = model.entity_representations[0]().detach().cpu().numpy()

No random seed is specified. This may lead to non-reproducible results.
Training epochs on cuda:0: 100%|██████████| 5/5 [03:04<00:00, 36.87s/epoch, loss=0.0461, prev_loss=0.0674]


In [16]:
len(embeddings)

332197

In [17]:
len(dat.entity_to_id)

332197

In [18]:
len(data.i2e)

341270

In [21]:
import numpy as np
len(np.unique(reorder))

332195

In [24]:
#from utils import extract_ents
from pyrdf2vec.graphs import KG
from pyrdf2vec.graphs import KG, Vertex
from kgbench.load import Data

from typing import List, Dict, Tuple

def extract_ents(data: Data):
    train_entities = []
    train_target = []
    for d in data.training:
        ent = data.i2e[d[0]][0]
        train_entities.append(ent)
        train_target.append(int(d[1]))

    test_entities = []
    test_taget = []
    for d in data.withheld:
        ent = data.i2e[d[0]][0]
        test_entities.append(ent)
        test_taget.append(int(d[1]))

    entities = train_entities + test_entities
    target = train_target + test_taget
    return train_entities, test_entities, train_target, test_taget
train_entities, test_entities, train_target, test_taget = extract_ents(data) 

In [27]:
data.withheld[:,0]

tensor([282346, 276422, 277418,  ..., 277014, 275650, 273554])